## 6章 章末問題

In [1]:
import pandas as pd
import statsmodels.api as sm

(1) accrualsData.csvには，year列の3月末に決算を迎えた企業の当期純利益 (NI)，営業活動によるキャッシュ・フロー (CFO)，期首時点の総資産額 (laggedAssets)の仮想データが収録されている。$t$年 ($t$ = 2007, 2008, $\ldots$, 2013)の各企業のアクルーアルズを(6.3)式の通りに計算し，その大きさに基づいて10個のポートフォリオを構築してみよう。その後，ポートフォリオ内の銘柄に対して等しくウェイトをかけて，$t$年7月から$t+1$年6月までの1年間運用をおこなうというアクルーアルズにもとづく取引戦略を実行したとしよう。2007年7月から2014年6月までの84ヶ月の運用結果をもとにして，各ポートフォリオのFF3アルファを推定してみよう。また，アクルーアルズが最も高い銘柄群をショートし，それが最も低い銘柄群をロングするというロング・ショート戦略を採用した場合のFF3アルファもあわせて推定してみよう。

In [3]:
# stockMonthly.csvの読み込み
stockMonthly = pd.read_csv('./data/stockMonthly.csv', parse_dates=['month'])
stockMonthly['month'] = stockMonthly['month'].dt.to_period('M')
# print(stockMonthly)
#        ticker    month  open  high   low  close   volume     share     return  industry  qme qbeme
# 0       A0001  1991-01  1571  1605  1503   1533   109323  19856748   4.285714         A  ME1   BM5 
# 1       A0001  1991-02  1503  1516  1457   1503   108565  19856748  -1.956947         A  ME1   BM5 

# FF3アルファ推定用にffMonthly.csvを読み込み
ffMonthly = pd.read_csv('./data/ffMonthly.csv', parse_dates=['month'])
ffMonthly['month'] = ffMonthly['month'].dt.to_period('M')
# print(ffMonthly)
##        month   RMRF   SMB   HML    RF
## 0    1990-07  20.67 -1.56 -5.16  0.68
## 1    1990-08 -13.69 -3.63  0.98  0.66

# accrualsを計算する元となるCSVデータの読み込み
# 年はPeriod型のYearに変換しておく(後の結合キーとなる)。
accruals = pd.read_csv('./data/accrualsData.csv', parse_dates=['year'])
accruals['year'] = accruals['year'].dt.to_period('Y')
# print(accruals)
#      ticker  year     NI    CFO  laggedAssets
# 0     A0001  2007  13386   8243       61189.0
# 1     A0002  2007  12485  10953      141546.0

# (6.3)式の通りaccrualsを計算する
accruals['accruals'] = (accruals['NI'] - accruals['CFO']) / accruals['laggedAssets']

# accrualsを10分位に分割する
labels = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
accruals['decile'] = accruals.groupby('year')['accruals'].apply(lambda x: pd.qcut(x, q=10, labels=labels).rename('decile'))
# print(accruals)
#      ticker  year     NI    CFO  laggedAssets  accruals decile
# 0     A0001  2007  13386   8243       61189.0  0.084051     10
# 1     A0002  2007  12485  10953      141546.0  0.010823      8


# 月別ticker別リターンデータ
df = stockMonthly[['month', 'ticker', 'return']].copy()

# df上のmonthが、t年7月〜t+1年6月にaccrualのt年データをmergeしたい。
# そのために、まずdf側にmonthのyearを取り出した新たな列yearをつくる。
# df.monthが6月以前のyearは前年のaccrualを結合するために1を減ずる。
df['year'] = df['month'].dt.asfreq('Y')
df.loc[df['month'].dt.month <= 6, 'year'] = df['year'] - 1

# df上の各tickerについて、t年7月〜t+1年6月にaccrualのt年データをmergeを実行。
df = df.merge(accruals[['ticker', 'year', 'decile']], on=['ticker', 'year'])
# print(df)
#           month ticker     return  year decile
# 0       2007-07  A0001  -0.925475  2007     10
# 1       2007-08  A0001  -5.703048  2007     10

# デシル別月別に等加重平均リターンを求めaccrualsの10分位ポートフォリオの完成
rp = df.groupby(['decile', 'month']).mean()
rp = rp.reset_index()
rp = rp[(rp['month']>='2007-7') & (rp['month']<='2014-6')]
# print(rp)
#     decile    month     return
# 0        1  2007-07   4.623807
# 1        1  2007-08  -4.579841

# decile=1をlong、decile=10をshortするポートフォリオの構築
decile1 = rp[rp['decile']==1]
decile10 = rp[rp['decile']==10]
rpLS = decile1.merge(decile10, on='month')
# print(rpLS)
#    decile_x    month   return_x decile_y   return_y
# 0         1  2007-07   4.623807       10   4.755723
# 1         1  2007-08  -4.579841       10  -5.748746
rpLS['return'] = rpLS['return_x'] - rpLS['return_y'] # 各月のロング・ショート戦略によるリターン
rpLS = rpLS[['month', 'return']]

def ols(d):
    # 各ポートフォリオの回帰モデル
    model = sm.OLS(d['RPRF'], sm.add_constant(d[['RMRF', 'SMB', 'HML']]))
    res = model.fit()
    coef = res.params[['const', 'RMRF', 'SMB', 'HML']]
    coef.index = ['alpha', 'b', 's', 'h']
    tval = res.tvalues  # t値
    tval.index = ['alpha_t', 'RMRF_t', 'SMB_t', 'HML_t']
    pval = res.pvalues  # p値
    pval.index = ['alpha_p', 'RMRF_p', 'SMB_p', 'HML_p']
    rsqr = pd.Series(res.rsquared, index=['R2'])
    return pd.concat([coef, tval, pval, rsqr])


# ポートフォリオ評価用のデータセットを作成する。
rp = rp.merge(ffMonthly[['month', 'RMRF', 'SMB', 'HML', 'RF']], on='month')
rp['RPRF'] = rp['return'] - rp['RF']
# print(rp)
#     decile    month     return  RMRF   SMB   HML   RF       RPRF
# 0        1  2007-07   4.623807  1.81  1.64  1.05  0.4   4.223807
# 1        2  2007-07   3.955725  1.81  1.64  1.05  0.4   3.555725

# ポートフォリオごとにFF3アルファを推定
res = rp.groupby('decile').apply(ols)
print('#### decile別の結果')
print(res)

# ロング・ショート戦略のポートフォリオ評価のデータセットを作成する。
rpLS = rpLS.merge(ffMonthly[['month', 'RMRF', 'SMB', 'HML', 'RF']], on='month')
rpLS['RPRF'] = rpLS['return']
# print(rpLS)
#       month    return  RMRF   SMB   HML    RF      RPRF
# 0   2007-07 -0.131917  1.81  1.64  1.05  0.40 -0.131917
# 1   2007-08  1.168906 -4.35 -0.62  2.86  0.42  1.168906

resLS = ols(rpLS)
print('#### ロング・ショート戦略の結果')
print(resLS)


#### decile別の結果
           alpha         b         s         h   alpha_t     RMRF_t  \
decile                                                                
1       0.358397  1.199258  0.938352  0.097586  1.965155  35.513239   
2       0.111298  1.040723  0.713477  0.234716  0.885038  44.694668   
3       0.193842  0.996249  0.659304  0.279398  1.715873  47.626766   
4      -0.031652  0.938980  0.594545  0.319104 -0.256219  41.050360   
5      -0.033136  0.963656  0.595924  0.362344 -0.294637  46.276196   
6      -0.019749  0.935237  0.606800  0.382169 -0.182669  46.717115   
7      -0.106261  0.949562  0.666921  0.345533 -0.818624  39.507413   
8      -0.042794  0.907560  0.739207  0.276507 -0.327823  37.546884   
9      -0.203069  0.954536  0.738263  0.299467 -1.581980  40.160065   
10     -0.203833  1.016900  0.893092  0.238102 -1.332460  35.900814   

            SMB_t     HML_t   alpha_p        RMRF_p         SMB_p  \
decile                                                        

In [49]:
# 全セルで計算した結果を表6.1のように整形する(各変数の上段に回帰係数を下段にt値を表示する)。
# 説明変数ごとに、回帰係数とt値の列を取得し、それぞれ統一して'coef'と't-val'という列名にしておく。
tbl_alpha = res[['alpha', 'alpha_t']].rename(columns={'alpha':'coef', 'alpha_t':'t-val'})
tbl_b = res[['b', 'RMRF_t']].rename(columns={'b':'coef', 'RMRF_t':'t-val'})
tbl_s = res[['s', 'SMB_t']].rename(columns={'s':'coef', 'SMB_t':'t-val'})
tbl_h = res[['h', 'HML_t']].rename(columns={'h':'coef', 'HML_t':'t-val'})
tbl_r2 = res[['R2']]  # 決定係数
# print(tbl_alpha)  # tbl_b, tbl_s, tbl_hも同じデータ構造
#             coef     t-val
# decile                    
# 1       0.358397  1.965155
# 2       0.111298  0.885038

# ここまでに用意した5つのDataFrameを横方向(axis=1)に連結(pd.concat)する。
# このような連結が可能なのは全ての表の行ラベル(index)が同じだからである。
# また、以下のように、連結する表を辞書としてconcat()に与えると、'alpha'や'b'などのキーが上位レベルのcolumns名となり、2段のマルチインデックスとなる。
tbl = pd.concat({'alpha':tbl_alpha, 'b':tbl_b, 's':tbl_s, 'h':tbl_h, 'R2':tbl_r2}, axis=1)
# print(tbl)
#            alpha                   b                    s             ...
#             coef     t-val      coef      t-val      coef      t-val   
# decile                                                                 
# 1       0.358397  1.965155  1.199258  35.513239  0.938352  14.395827  ... 
# 2       0.111298  0.885038  1.040723  44.694668  0.713477  15.874270  ... 

# 行indexであるAccrualsのデシル名を表6.1に合わせて変更する。
tbl.index = ['Lowest Accruals', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'Highest Accruals']

# Long/Shortの結果は以下で求める。
# tbl_lsはSeriesである。
tbl_ls = resLS[['alpha', 'alpha_t', 'b', 'RMRF_t', 's', 'SMB_t', 'h', 'HML_t', 'R2']]
tbl_ls.name = 'Long-Short'
# print(tbl_ls)
# alpha      0.562230
# alpha_t    2.655218
#    :           :
# R2         0.226269
# Name: Long-Short, dtype: float64

# Seriesのインデックスをtblのcolumnsと合わせておく。
tbl_ls.index = tbl.columns

# tblを転置して、R2の列を加え、小数点以下を3桁に四捨五入すればできあがり。
tbl = pd.concat([tbl.T, tbl_ls], axis=1).round(3)
print(tbl)
tbl  # HTMLで表示

             Lowest Accruals      D2      D3      D4      D5      D6      D7  \
alpha coef             0.358   0.111   0.194  -0.032  -0.033  -0.020  -0.106   
      t-val            1.965   0.885   1.716  -0.256  -0.295  -0.183  -0.819   
b     coef             1.199   1.041   0.996   0.939   0.964   0.935   0.950   
      t-val           35.513  44.695  47.627  41.050  46.276  46.717  39.507   
s     coef             0.938   0.713   0.659   0.595   0.596   0.607   0.667   
      t-val           14.396  15.874  16.329  13.466  14.826  15.703  14.375   
h     coef             0.098   0.235   0.279   0.319   0.362   0.382   0.346   
      t-val            1.062   3.706   4.911   5.129   6.397   7.019   5.286   
R2    R2               0.941   0.962   0.966   0.956   0.965   0.966   0.952   

                 D8      D9  Highest Accruals  Long-Short  
alpha coef   -0.043  -0.203            -0.204       0.562  
      t-val  -0.328  -1.582            -1.332       2.655  
b     coef    0.908

Lowest Accruals      D2      D3      D4      D5      D6      D7  \
alpha coef             0.358   0.111   0.194  -0.032  -0.033  -0.020  -0.106   
      t-val            1.965   0.885   1.716  -0.256  -0.295  -0.183  -0.819   
b     coef             1.199   1.041   0.996   0.939   0.964   0.935   0.950   
      t-val           35.513  44.695  47.627  41.050  46.276  46.717  39.507   
s     coef             0.938   0.713   0.659   0.595   0.596   0.607   0.667   
      t-val           14.396  15.874  16.329  13.466  14.826  15.703  14.375   
h     coef             0.098   0.235   0.279   0.319   0.362   0.382   0.346   
      t-val            1.062   3.706   4.911   5.129   6.397   7.019   5.286   
R2    R2               0.941   0.962   0.966   0.956   0.965   0.966   0.952   

                 D8      D9  Highest Accruals  Long-Short  
alpha coef   -0.043  -0.203            -0.204       0.562  
      t-val  -0.328  -1.582            -1.332       2.655  
b     coef    0.908   0.955             1.017       0.182  
      t-val  37.547  40.160            35.901       4.651  
s     coef    0.739   0.738             0.893       0.045  
      t-val  15.844  16.092            16.335       0.598  
h     coef    0.277   0.299             0.238      -0.141  
      t-val   4.206   4.632             3.091      -1.318  
R2    R2      0.948   0.954             0.944       0.226

---
(2) コード 6.7 について，rolling().corr() をもちいて実装してみよう。

---

In [3]:
# データの選択

# 銘柄ごとの日次リターンデータの読み込みと、計算に必要な列の選択 (コード6.2再掲)
stockDaily = pd.read_csv('./data/stockDaily.csv', parse_dates=['date'])
stockDaily['date'] = stockDaily['date'].dt.to_period('D')
stockDaily = stockDaily[['ticker','date','open','high','low', 'close', 'volume']]
print(stockDaily)
#          ticker        date  open  high   low  close  volume
# 0         A0001  1991-01-04  1411  1498  1411   1457    1127
# 1         A0001  1991-01-07  1456  1456  1456   1456    1863

## 図6-8の入力データ(1)の作成: ds1  (コード6.6再掲)
# Aではじまるtickerの終値系列データをDataFrameとして作成
ds1 = stockDaily[['date', 'ticker', 'close']].copy()
ds1 = ds1[ds1['ticker'].str[0] == 'A']
print(ds1)
#               date ticker  close
# 0       1991-01-04  A0001   1457
# 1       1991-01-07  A0001   1456

         ticker        date  open  high   low  close  volume
0         A0001  1991-01-04  1411  1498  1411   1457    1127
1         A0001  1991-01-07  1456  1456  1456   1456    1863
2         A0001  1991-01-08  1424  1441  1424   1441    8084
3         A0001  1991-01-09  1437  1437  1437   1437    7542
4         A0001  1991-01-10  1401  1451  1384   1424     986
...         ...         ...   ...   ...   ...    ...     ...
12005999  Z0137  2014-12-24   645   645   645    645  237742
12006000  Z0137  2014-12-25   640   643   628    643   77924
12006001  Z0137  2014-12-26   649   649   645    645  110739
12006002  Z0137  2014-12-29   635   653   618    631  226144
12006003  Z0137  2014-12-30   644   647   644    647  286380

[12006004 rows x 7 columns]
              date ticker  close
0       1991-01-04  A0001   1457
1       1991-01-07  A0001   1456
2       1991-01-08  A0001   1441
3       1991-01-09  A0001   1437
4       1991-01-10  A0001   1424
...            ...    ...    ...
351175  

In [4]:
# date×tickerの終値行列の作成 
ds2 = ds1.pivot_table(index='date', columns='ticker', values='close')
print(ds2)
# ticker       A0001    A0002   A0003   A0004   A0005    A0006   A0007 ...
# date                                                                
# 1991-01-04  1457.0      NaN     NaN  4675.0  1560.0  10931.0     NaN ...
# 1991-01-07  1456.0      NaN     NaN  4582.0  1558.0  10934.0     NaN ...
# 1991-01-08  1441.0      NaN     NaN  4747.0  1583.0  11021.0     NaN ...


ticker       A0001    A0002   A0003   A0004   A0005    A0006   A0007   A0008  \
date                                                                           
1991-01-04  1457.0      NaN     NaN  4675.0  1560.0  10931.0     NaN  2087.0   
1991-01-07  1456.0      NaN     NaN  4582.0  1558.0  10934.0     NaN  2099.0   
1991-01-08  1441.0      NaN     NaN  4747.0  1583.0  11021.0     NaN  2074.0   
1991-01-09  1437.0      NaN     NaN  4642.0  1549.0  10824.0     NaN  2070.0   
1991-01-10  1424.0      NaN     NaN  4614.0  1523.0  10532.0     NaN  2108.0   
...            ...      ...     ...     ...     ...      ...     ...     ...   
2014-12-24  1488.0  19827.0  2215.0   899.0  1355.0  11144.0  2148.0  2225.0   
2014-12-25  1505.0  19773.0  2191.0   890.0  1351.0  11083.0  2109.0  2218.0   
2014-12-26  1533.0  19856.0  2215.0   894.0  1361.0  11125.0  2104.0  2216.0   
2014-12-29  1480.0  19668.0  2124.0   901.0  1333.0  11085.0  2045.0  2190.0   
2014-12-30  1549.0  19779.0  2133.0   91

In [5]:
# rolling.corr()を用いた方法
# 一旦、全ての日付の相関行列を生成してしまうので、
# 銘柄数が多くなるとメモリエラーとなる可能性がある。

def count(d):
    # NaNを含むtickerとの相関係数は行/列ともに全てNaNになっているので、
    # dropnaを行方向と列方向で実行することで削除する。
    d = d.dropna(how='all').dropna(how='all',axis=1)
    # ticker                A0001     A0004     A0005     A0006 ...
    # date       ticker                                        
    # 1991-01-18 A0001   1.000000 -0.086346 -0.395261 -0.097276
    #            A0004  -0.086346  1.000000  0.690526  0.749451
    #            A0005  -0.395261  0.690526  1.000000  0.850203
    #            A0006  -0.097276  0.749451  0.850203  1.000000
            
    # 相関行列の全ての値について0.9以上かどうかのbool値に直し
    # astype(int)でFalseを0、Trueを1に変換し、それらの合計を計算している。
    c = (d.values >= 0.9).astype(int).sum()
    
    # 相関行列は上三角行列と下三角行列が対称なので、
    # 対角成分の数(len(d))を除いて2で割っている。
    return (c - len(d)) / 2

# 10日窓での相関行列の計算
# 10日窓にNaNが１つでもあれば、その列との相関係数は全てNaNと計算されることに注意する。
# DataFrameのcorr()メソッドのようにmin_periodsを指定できない。
corr_matrix = ds2.rolling(window=10).corr()
corr = corr_matrix.groupby('date').apply(count)
corr = corr.dropna()
corr.name = 'corr'
print(corr)

date
1991-01-04     0.0
1991-01-07     0.0
1991-01-08     0.0
1991-01-09     0.0
1991-01-10     0.0
              ... 
2014-12-24    46.0
2014-12-25    52.0
2014-12-26    68.0
2014-12-29    40.0
2014-12-30    46.0
Freq: D, Name: corr, Length: 5903, dtype: float64
